In [1]:
!pip install pdf2image paddleocr layoutparser transformers torch torchvision
!apt install poppler-utils  # or install poppler on Windows (add to PATH)

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
pip install paddlepaddle-gpu==2.5.0.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement paddlepaddle-gpu==2.5.0.post117 (from versions: 2.6.0, 2.6.1, 2.6.2)
ERROR: No matching distribution found for paddlepaddle-gpu==2.5.0.post117


In [3]:
pip install paddlepaddle==2.5.0 -i https://pypi.org/simple


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement paddlepaddle==2.5.0 (from versions: 2.6.0, 2.6.1, 2.6.2, 3.0.0b0, 3.0.0b1, 3.0.0b2, 3.0.0rc0, 3.0.0rc1, 3.0.0)
ERROR: No matching distribution found for paddlepaddle==2.5.0


In [4]:
import paddle
print(paddle.__version__)
print("CUDA Available:", paddle.is_compiled_with_cuda())



ModuleNotFoundError: No module named 'paddle'

In [ ]:
import os
from pdf2image import convert_from_path
from paddleocr import PaddleOCR
import torch
from PIL import Image
from transformers import LayoutLMv3Processor, LayoutLMv3ForTokenClassification
import matplotlib.pyplot as plt


# --- Load Models ---
ocr = PaddleOCR(use_angle_cls=True, lang='en')
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base")
model.eval()

# --- Convert PDF to Images ---
def pdf_to_images(pdf_path, dpi=300):
    return convert_from_path(pdf_path, dpi=dpi)

# --- Run PaddleOCR ---
def extract_ocr_data(image):
    result = ocr.ocr(image, cls=True)
    words, boxes = [], []
    for line in result[0]:
        txt = line[1][0]
        box = line[0]
        x_min = min([pt[0] for pt in box])
        y_min = min([pt[1] for pt in box])
        x_max = max([pt[0] for pt in box])
        y_max = max([pt[1] for pt in box])
        words.append(txt)
        boxes.append([x_min, y_min, x_max, y_max])
    return words, boxes

# --- Normalize boxes to 0-1000 range ---
def normalize_box(box, width, height):
    return [
        int(1000 * box[0] / width),
        int(1000 * box[1] / height),
        int(1000 * box[2] / width),
        int(1000 * box[3] / height),
    ]

# --- Process with LayoutLMv3 ---
def run_layoutlmv3(image_path):
    image = Image.open(image_path).convert("RGB")
    words, boxes = extract_ocr_data(image)

    width, height = image.size
    norm_boxes = [normalize_box(b, width, height) for b in boxes]

    encodings = processor(image, words, boxes=norm_boxes, return_tensors="pt", truncation=True, padding="max_length")
    with torch.no_grad():
        outputs = model(**encodings)
        predictions = outputs.logits.argmax(-1).squeeze().tolist()
        tokens = processor.tokenizer.convert_ids_to_tokens(encodings.input_ids.squeeze().tolist())

    print("\n📄 Structured Output:")
    for word, label_id in zip(words, predictions):
        label = model.config.id2label[label_id]
        if label != "O":  # Skip 'Other'
            print(f"{label}: {word}")

# --- MAIN ---
if __name__ == "__main__":
    pdf_path = r"C:\Users\HP\Desktop\Bank Project\Balraj\Bank Statement Nov-Jan 2019.pdf"
    images = pdf_to_images(pdf_path)

    for i, img in enumerate(images):
        img_path = f"page_{i}.png"
        img.save(img_path)
        print(f"\n🔍 Processing Page {i+1}")
        run_layoutlmv3(img_path)


ModuleNotFoundError: No module named 'paddle'